### Modélisation des données du Défi IA 2021-2022

Dans ce calepin, nous décrivons quelques éléments de modélisation des données du  Défi IA 2021-2022. Il est suggéré quelques pistes pour réaliser des statistiques descriptives de ces données ainsi qu'une approche de prévision du cumul de pluie à l'aide de la régression linéaire. Des codes en Python sont également proposés. Le but de calepin est également de vous aider à initier la rédaction d'un rapport sur votre travail dans ce Défi IA.

#### Données disponibles sur les stations de mesure

Sur les années 2016 et 2017, données d'apprentissage sur $N$ stations météorologiques dont on dispose des coordonnées spatiales (latitude et longitude).

Pour chaque station $1 \leq i \leq N$, on dispose des mesures suivantes :

**Variables explicatives** : mesure de $p$ variables $X_{ijt} = (X_{ijt}^{(k)})_{1 \leq k \leq p} \in \mathbb{R}^{p}$ pour la station $i$, le jour $j$ (variable non-ordonnée car non-disponible dans l'ensemble test) et l'heure $t \in \{0,\ldots,23 \}$ (variable ordonnée disponible dans l'ensemble test). Les mesures sont

- 'ff' : *inclure une description*
- 't' : *inclure une description*
- 'td' : *inclure une description*
- 'hu' : *humidité*
- 'dd' : *inclure une description*
- 'precip' : *cumul de pluie sur une heure en ml*

On peut également ajouter une variable sur le mois de l'année car cette information est disponible dans l'ensemble test.


**Variable à expliquer/prédire** : cumul de pluie $Y_{ij}$ sur une journée au jour $j+1$ dans la station $i$ à partir des données disponibles au jour $j$. Dans l'ensemble d'apprentissage, on dipose en fait de la variable $Y_{ijt}$ cumul de pluie  sur une journée au jour $j+1$ dans la station $i$ et àl'heure $t$. De façon évidente on a que (avec $T=23$)
$$
Y_{ij} = \sum_{t = 0}^{T} Y_{ijt}
$$

**Travail préliminaire** : proposer une analyse descriptive de ces données : boxplot, histogramme uni-varié, ACP pour étude des corrélation entre variables explicatives, etc...

**Modèles linéaires possibles de prévision du cumul de pluie** : 

*Modèle global temps par temps*

$$
Y_{ijt} = \theta_{0}^{t} + \sum_{k = 1}^{p} \theta_{k}^{t}X_{ijt}^{(k)} + \varepsilon_{ijt}
$$

et prévision par $\hat{Y}_{ij} = \sum_{t = 0}^{T} \hat{Y}_{ijt} $ où $\hat{Y}_{ijt} = \hat{\theta}_{0}^{t} + \sum_{k = 1}^{p} \hat{\theta}_{k}^{t}X_{ijt}^{(k)}$

*Modèle par station et temps par temps*

$$
Y_{ijt} = \theta_{0,i}^{t} + \sum_{k = 1}^{p} \theta_{k,i}^{t}X_{ijt}^{(k)} + \varepsilon_{ijt}
$$

où les cofficients du modèle linéaire varient selon la station de mesure.

*Modèle global avec agrégation du temps*

$$
Y_{ij} = \theta_{0} + \sum_{t = 0}^{T}  \sum_{k = 1}^{p} \theta_{k}^{t}X_{ijt}^{(k)} + \varepsilon_{ij}
$$

et bien d'autres modèles sont possibles !

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [ ]:
import os
os.chdir('/content/drive/My Drive/Données Massives/')

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import display


In [ ]:
import pandas as pd
import datetime
import seaborn as sns
import numpy as np

# Suppression des messages d'erreur liés à des besoins de mise à jour de syntaxe en Python
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

warnings.filterwarnings("ignore")

In [ ]:
# Lecture des données de l'ensemble d'apprentissage 

path = 'defi-ia-2022/Train/Train/X_station_train.csv'
first_date = datetime.datetime(2016,1,1)    
last_date = datetime.datetime(2017,12,31)

# Read the ground station data
def read_gs_data(fname):
    gs_data = pd.read_csv(fname,parse_dates=['date'],infer_datetime_format=True)
    gs_data = gs_data.sort_values(by=["number_sta","date"])
    return gs_data

x = read_gs_data(path)
x['number_sta']=x['number_sta'].astype('category')

# Tri par station puis par datea
x = x.sort_values(['number_sta','date'])
x

In [ ]:
# Ajout des variables jour et heure
Xtrain = x
split_Id = Xtrain['Id'].str.split(pat="_", expand = True)
split_Id = split_Id.rename(columns={0: "number_sta_2", 1: "day", 2: "hour"})
Xtrain['number_sta_2'] = split_Id['number_sta_2']
Xtrain['day'] = split_Id["day"]
Xtrain['hour'] = split_Id["hour"]
Xtrain = Xtrain.drop("number_sta_2",axis=1)
display(Xtrain) 

In [ ]:
#Ajout de mois 

Xtrain['month'] = pd.DatetimeIndex(Xtrain['date']).month
display(Xtrain) 

In [ ]:
################################################
# Inclure une analyse desciptive des données ! #
################################################

Barplot des précipitations

In [ ]:
# bins = [0,1,2,4,5,6,7,8,9,10,20,30]
# data_bar_precip = Xtrain.groupby(pd.cut(Xtrain['precip'], bins=bins)).count()
# data_bar_precip['precip'].plot.bar()

In [ ]:
# Xtrain.boxplot()

In [ ]:
# corr_Xtrain = Xtrain.corr(method='pearson')
# corr_Xtrain
# plt.matshow(corr_Xtrain)
# plt.show()

Matrice de corrélation de Xtrain
On retrouve une corrélation négative entre la température et l'humidité ce qui semble logique
On constate aussi une forte corrélation positive entre la température et td

In [ ]:
# sns.heatmap(corr_Xtrain,annot=True)
# plt.show()

In [ ]:
#On rajoute deux variables 
#precip_bool: 1: il pleut 0: il ne pleut pas
#precip _ategories: 5 categories 0, 0.2, 0.4 , 0.6, 0.8+


#Xtrain_2017['col2'] = pd.cut(df['col1'], bins=[0, 10, 50, float('Inf')], labels=['xxx', 'yyy', 'zzz'])

#Xtrain_2017



In [ ]:
# Exemple d'implémentation du modèle linéaire par station et temps par temps mais avec
# des coefficients qui ne dépendent pas du temps
# Imputation de données manquantes par la médiane des variables

################################################################################################
# Attention, une erreur de mosélisation s'est glissée dans ce code, saurez-vous la retrouver ? #
################################################################################################

from sklearn.linear_model import LinearRegression

def regression_bystation (x,num_station):

    X = x[x['number_sta']==num_station]
    Y = X[{"date","precip"}]
    Y.set_index('date',inplace = True) 

    X = X[{"date","ff","t","td","hu","dd"}]
    X.set_index('date',inplace = True)

    # Imputation des valeurs manquantes
    median_t = X['t'].median()
    X['t'] = X['t'].fillna(median_t)

    median_ff = X['ff'].median()
    X['ff'] = X['ff'].fillna(median_ff)

    median_td = X['td'].median()
    X['td'] = X['td'].fillna(median_td)

    median_hu = X['hu'].median()
    X['hu'] = X['hu'].fillna(median_hu)

    median_dd = X['dd'].median()
    X['dd'] = X['dd'].fillna(median_dd)

    median_pre = Y['precip'].median()
    Y['precip'] = Y['precip'].fillna(median_pre)

    lr= LinearRegression(normalize=False)
    lr.fit(X, Y)

    return(lr)

ACP normée avec deux composantes principales


In [ ]:
from sklearn.preprocessing import StandardScaler

# Xtrain = Xtrain.fillna(method='ffill')
# features = ['ff','t','td','hu','dd']

# X_acp = Xtrain.loc[:,features].values
# X_acp = StandardScaler().fit_transform(X_acp)


# Y_acp = Xtrain.loc[:,'precip'].values



In [ ]:
from sklearn.decomposition import PCA

# pca = PCA(n_components = 5)

# principalComponents = pca.fit_transform(X_acp)

# principalDf = pd.DataFrame(data = principalComponents
#              , columns = ['principal component 1', 'principal component 2','principal component 3','principal component 4','principal component 5'])

# finalDf = pd.concat([principalDf, Xtrain[['precip']]], axis = 1)



On récupère les valeurs propres de nos composantes principales

In [ ]:
# pca.explained_variance_

Notre première composante principale explique 37% de l'inertie tandis que la seconde composante principale explique 25 % de l'inertie.

In [ ]:
# pca.explained_variance_ratio_

Visualisation des valeurs propres de nos composantes principales. 

In [ ]:

# sns.set_theme(style='darkgrid')
# graph_variance = sns.lineplot(x=['1','2','3','4','5'],y = pca.explained_variance_)
# graph_variance.axhline(1,color= 'red')

On constate que les composantes principales 1 et 2 ont des valeurs propres supérieurs à 1. 
Tandis que la troisième composante principale à une valeur propre très proche de 1 c'est pourquoi on décide aussi de la garder pour la suite selon la règle de Kaiser.


In [ ]:
# display(finalDf)

S'il pleut, la variable class prend comme valeur 1, sinon elle prend comme valeur 2.


In [ ]:
# finalDf['class'] = np.where(finalDf['precip'] == 0,1,2)
# display(finalDf)



Scatter plot avec les deux premières composantes principales

In [ ]:
# sns.scatterplot(data = finalDf,x= 'principal component 1', y = 'principal component 2', hue='class')

En 3 dimensions cela devient illisible.

In [ ]:
# fig = plt.figure()
# fig.set_size_inches(15, 10.5)
# ax = fig.add_subplot(111, projection='3d')
# x = np.array(finalDf['principal component 1'])
# y = np.array(finalDf['principal component 2'])
# z = np.array(finalDf['principal component 3'])
# ax.scatter(x,y,z, marker="s", c=finalDf["class"], s=20, cmap="RdBu")
# ax.set_xlabel('Principal Component 1')
# ax.set_ylabel('Principal Component 2')
# ax.set_zlabel('Principal Component 3')
# ax.legend()
# plt.show()

On va tenter d'ajouter une variable explicative ayant une bonne corrélation avec la variable precip.Pour cela, on va utiliser l'algorithme knn qui va nous dire s'il pleut ou pas.
La nouvelle variable knn qui sera crée prendra en valeur la prédiction de la classe donnée par l'algorithme knn. ( soit il pleut soit il ne pleut pas)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
# knn = KNeighborsClassifier(n_neighbors=10)



In [ ]:
Xtrain

In [ ]:
Xtrain = Xtrain.fillna(method='ffill')
Xtrain['class'] = np.where(Xtrain['precip'] == 0,1,2)


X = Xtrain.drop(['date','class','Id'],axis=1)
Y = Xtrain['class']
X['day'] = X['day'].astype('category')
X['hour'] = X['hour'].astype('category')

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.3, random_state=4,stratify=Y)

In [ ]:
# #Xtrain['year'] = Xtrain[]
# Xtrain = Xtrain.fillna(method='ffill')
# Xtrain_gp = Xtrain.groupby(by=['number_sta','day'])



On standardise nos données

In [ ]:
#knn.fit(X_train,y_train)
#predictions = knn.predict(X_test)
#accuracy_score(y_test,predictions)

In [ ]:
# knn.fit(X_train,y_train)

In [ ]:

accuracy_score(y_test,np.repeat(1,y_test.shape[0]))


En ne prédisant que des 1 (i.e jours où il pleut), on obtient une précision de 90%

In [ ]:
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
# random_forest = RandomForestClassifier()
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.3, random_state=4,stratify=Y)
# ss = StandardScaler()

# X_train = ss.fit_transform(X_train)
# X_test = ss.transform(X_test)

# random_forest.fit(X_train,y_train)


In [ ]:
# predictions = random_forest.predict(X_test)
# accuracy_score(y_test,predictions)

In [ ]:
# np.unique(predictions,return_counts=True)


In [ ]:
# np.unique(y_test,return_counts=True)



In [ ]:
# predictions = knn.predict(X_test)
# accuracy_score(y_test,predictions)

On obtient une précision de 91% avec knn (k=10)


On va maintenant tenter d'améliorer ce score en utilisant la méthode de validation croisée avec knn et en prenant une grille de valeur de k afin de choisir le k qui maximise la précision de notre modèle.


In [ ]:
# from sklearn.model_selection import GridSearchCV 
# knn2 = KNeighborsClassifier() 
# param_grid = {"n_neighbors" : np.arange(1, 25)}
# knn_cv = GridSearchCV(knn2, param_grid, cv=3)
# knn_cv.fit(X, Y)

In [ ]:
# from sklearn.model_selection import GridSearchCV 
# knn2 = KNeighborsClassifier() 
# param_grid = {"n_neighbors" : np.arange(1, 25)}

In [ ]:
X = Xtrain
Y = X[{"date","precip"}]
Y.set_index('date',inplace = True) 

# Imputation des valeurs manquantes
#X = X[{"date","ff","t","td","hu","dd"}]
#X.set_index('date',inplace = True)

In [ ]:
Xtrain

In [ ]:
# Réorganisation des données pour modèle avec aggrégation dans le temps
X["number_sta_day"] = Xtrain['number_sta'].astype(str) + '_' + Xtrain['day'].astype(str)
X

In [ ]:
X.drop(['class','Id','number_sta'],axis=1,inplace= True)

In [ ]:
X['ff_idx'] = 'ff_' + X["hour"].astype(str)
X['t_idx'] = 't_' + X["hour"].astype(str)
X['td_idx'] = 'td_' + X["hour"].astype(str)
X['hu_idx'] = 'hu_' + X["hour"].astype(str)
X['dd_idx'] = 'dd_' + X["hour"].astype(str)
X['precip_idx'] = 'precip_' + X["hour"].astype(str)

In [ ]:
ff = X.pivot(index='number_sta_day',columns='ff_idx',values='ff')
t = X.pivot(index='number_sta_day',columns='t_idx',values='t')
td = X.pivot(index='number_sta_day',columns='td_idx',values='td')
hu = X.pivot(index='number_sta_day',columns='hu_idx',values='hu')
dd = X.pivot(index='number_sta_day',columns='dd_idx',values='dd')
precip = X.pivot(index='number_sta_day',columns='precip_idx',values='precip')

In [ ]:
X_reshape = pd.concat([ff ,t,td,hu,dd,precip],axis=1)

In [ ]:
X_reshape.reset_index(inplace=True)
X_reshape

In [ ]:
# Données de cumul de pluie
fname = 'defi-ia-2022/Train/Train/Y_train.csv'
param = 'Ground_truth'  #weather parameter name in the file ('Ground_truth' about Y and 'Prediction' about baseline)
data = pd.read_csv(fname, parse_dates=['date'], infer_datetime_format=True)
data['number_sta'] = data['number_sta'].astype('category')
display(data)

In [ ]:
Y = data[["Ground_truth","Id"]]
Y.rename(columns = {"Ground_truth":"Y","Id":"number_sta_day"},inplace=True)
Y.dropna(inplace=True)
display(Y)

In [ ]:
all_data = pd.merge(X_reshape,Y)

display(all_data)
all_data.dropna(inplace=True)

In [ ]:

Y = all_data['Y']
X = all_data.drop("Y",axis=1)
X

In [ ]:
X['day'] = X['number_sta_day'].str.rpartition("_")[2]
X['day'] = X['day'].astype('int')
X['day'] = X['day'] % 365

In [ ]:
X['number_sta'] = X['number_sta_day'].str.rpartition('_')[0]
X['number_sta'] = X['number_sta'].astype('int64')

In [ ]:
X['season'] =pd.cut(X['day'],[0,60,151,243,334], labels=['Hiver','Printemps','Ete', 'Automne'],ordered=False)
X['season'] = X['season'].fillna('Hiver')
X['season']

In [ ]:
X['month'] = X['day'] / 30
X['month'] = X['month'].astype('int')


In [ ]:
X.reset_index(inplace=True)
X.drop('index',axis=1,inplace=True)

In [ ]:
X['month'] = X['month'].astype('category')
X['season'] = X['season'].astype('category')
X['day'] = X['day'].astype('category')

Ajoutons les coordonnées des stations

In [ ]:
#open the file with station coordinates (latitude/longitude)

coords_fname  = 'defi-ia-2022/Other/Other/stations_coordinates.csv'
coords = pd.read_csv(coords_fname)
coords['number_sta'] = coords['number_sta'].astype('category')
display(coords)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
len(np.unique(X['number_sta']))

In [ ]:
coords.dtypes

In [ ]:
X =pd.merge(X,coords,how='left',on='number_sta')
X['number_sta'] = X['number_sta'].astype('category')
X

In [ ]:
Y =pd.DataFrame(Y)
Y.reset_index(inplace=True)
Y.drop('index',inplace = True,axis=1)

In [ ]:
Y

In [ ]:
#X['class'] = np.where(Y['Y'] == 0,1,2)
#Y = X.loc[:,X.columns.str.startswith('precip')].sum(axis=1)
Class =np.where(Y['Y'] == 0,1,2)
Class = pd.DataFrame(Class)
Class.reset_index(inplace=True)
Class.drop('index',inplace=True,axis=1)

In [ ]:
Class

On souhaite enrichir notre base de données en ajoutant une nouvelle variable knn qui prend comme valeur la classe associé à la à chaque station en fonction de leur coordonnées 

Utilisons KNN

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder,OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
split = ShuffleSplit(n_splits=1, test_size=0.3,random_state = 4)
split.get_n_splits(X, Class)
train_index, test_index = next(split.split(X, Class)) 

X_train,X_test = X.loc[train_index], X.loc[test_index]
Y_train, Y_test = Class.loc[train_index], Class.loc[test_index]


In [ ]:
transformer = ColumnTransformer(
                     [
                         ('transform_name_categories', OrdinalEncoder(), make_column_selector(dtype_include=object)),
                         ('transformer_name_for_numerical', StandardScaler(), make_column_selector(dtype_include=np.number))
                     ]
                 )
X_train = transformer.fit_transform(X_train)
X_test = transformer.fit_transform(X_test)


In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train,Y_train)

predictions = knn.predict(X_test)
accuracy_score(Y_test,predictions)


In [ ]:

split = ShuffleSplit(n_splits=1, test_size=0.3,random_state = 4)
split.get_n_splits(X, Class)
train_index, test_index = next(split.split(X, Class)) 

X_train,X_test = X.loc[train_index], X.loc[test_index]
Y_train, Y_test = Class.loc[train_index], Class.loc[test_index]


In [ ]:
random_forest = RandomForestClassifier(oob_score=True)

transformer = ColumnTransformer(
                     [
                         ('transform_name_categories', OrdinalEncoder(), make_column_selector(dtype_include=object)),
                         ('transformer_name_for_numerical', StandardScaler(), make_column_selector(dtype_include=np.number))
                     ]
                 )
X_train = transformer.fit_transform(X_train)
X_test = transformer.fit_transform(X_test)

X_train
random_forest.fit(X_train,Y_train)


In [ ]:
predictions = random_forest.predict(X_test)
accuracy_score(Y_test,predictions)

On récupère les features importantes

In [ ]:
# Get numerical feature importances
importances = list(random_forest.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(X.columns, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]

Utilisons Support Vector Machine (SVM) afin de classifier les jours où il pleut

In [ ]:
# from sklearn import svm
# svm_model = svm.SVC(kernel = 'linear')
# svm_model.fit(X_train,Y_train)
# predictions = svm_model.predict(X_test)
# accuracy_score(Y_test,predictions)

In [ ]:
# predictions

In [ ]:
X_test = pd.DataFrame(X_test)
X_test['index_original'] = test_index
X_test

In [ ]:
index_pluie =np.where(predictions==2)

In [ ]:
X_pluie = X_test.loc[index_pluie]
X_pluie

In [ ]:
X_pluie.shape

In [ ]:
Y_test = pd.DataFrame(Y_test)
Y_test

In [ ]:
Y = pd.DataFrame(Y)
Y.reset_index(inplace=True)
Y.drop('index',axis=1,inplace=True)

In [ ]:
Y_pluie = Y.loc[X_pluie['index_original']]
pd.DataFrame(Y_pluie)



In [ ]:
Y_pluie = pd.DataFrame(Y_pluie)
Y_pluie = Y_pluie.reset_index()


In [ ]:
Y_pluie.drop('index',axis=1,inplace=True)

In [ ]:
Y_pluie

In [ ]:
sum(Y_pluie['Y']==0)

Après avoir prédit les jours où il pleut, on utilise un modèle prédictif afin de déterminer le cumul de pluie par jour

In [ ]:
index_original = X_pluie['index_original']
X_pluie.drop('index_original',axis=1,inplace=True)

In [ ]:
X_pluie.reset_index(inplace=True)

In [ ]:
X_pluie.drop('index',axis=1,inplace=True)

In [ ]:
Y_pluie

In [ ]:
split = ShuffleSplit(n_splits=1, test_size=0.3,random_state = 4)
split.get_n_splits(X_pluie, Y_pluie)
train_index, test_index = next(split.split(X_pluie, Y_pluie)) 

X_train,X_test = X_pluie.loc[train_index], X_pluie.loc[test_index]
Y_train, Y_test = Y_pluie.loc[train_index], Y_pluie.loc[test_index]


Utilisons une régression linéaire


In [ ]:
from sklearn import linear_model, metrics

reg = linear_model.LinearRegression()

reg.fit(X_train,Y_train)



In [ ]:
predictions = reg.predict(X_test)
predictions

In [ ]:
np.mean(np.abs(predictions-Y_test)/(Y_test+1))*100

In [ ]:
metrics.mean_absolute_error(predictions,Y_test)

Random Forest Regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(random_state=4)
regressor.fit(X_train, Y_train)
predictions = regressor.predict(X_test)

In [ ]:
new_predictions = [[i] for i in predictions]
new_predictions = np.asarray(new_predictions)

In [ ]:
np.mean(np.abs(new_predictions-Y_test)/(Y_test+1))*100

In [ ]:
metrics.mean_absolute_error(predictions,Y_test)

Quantile regression

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_pinball_loss, mean_squared_error


all_models = {}
common_params = dict(
    learning_rate=0.05,
    n_estimators=100,
    max_depth=2,
    min_samples_leaf=9,
    min_samples_split=9,
)
for alpha in [0.05]:
    gbr = GradientBoostingRegressor(loss="quantile", alpha=alpha, **common_params)
    all_models["model_1"] = gbr.fit(X_train, Y_train)

In [ ]:
predictions = all_models['model_1'].predict(X_test)


In [ ]:
quantile_regression_pred = [[i] for i in predictions]
quantile_regression_pred = np.asarray(quantile_regression_pred)

In [ ]:
np.mean(np.abs(quantile_regression_pred-Y_test)/(Y_test+1))*100

Prédictions sur données Test

In [ ]:
# Lecture des données de l'ensemble test

path = 'defi-ia-2022/Test/Test/X_station_test.csv'


with open(path, encoding="utf8", errors='ignore') as f:
  contents = f.read()
Xtest = pd.read_csv(path)
display(Xtest)
#Xtest['number_sta']=Xtest['number_sta'].astype('category')

# Tri par station puis par datea
#Xtest = Xtest.sort_values(['number_sta','date'])
#Xtest

In [ ]:
split_Id = Xtest['Id'].str.split(pat="_", expand = True)
split_Id = split_Id.rename(columns={0: "number_sta_2", 1: "day", 2: "hour"})
Xtest['number_sta_2'] = split_Id['number_sta_2']
Xtest['day'] = split_Id["day"]
Xtest['hour'] = split_Id["hour"]
Xtest['Id'] = split_Id['number_sta_2'] + "_" + split_Id['day']
Xtest = Xtest.drop("number_sta_2",axis=1)
Xtest.fillna(method='bfill',inplace=True)
display(Xtest)

In [ ]:
Xtest['ff_idx'] = 'ff_' + Xtest["hour"].astype(str)
Xtest['t_idx'] = 't_' + Xtest["hour"].astype(str)
Xtest['td_idx'] = 'td_' + Xtest["hour"].astype(str)
Xtest['hu_idx'] = 'hu_' + Xtest["hour"].astype(str)
Xtest['dd_idx'] = 'dd_' + Xtest["hour"].astype(str)
Xtest['precip_idx'] = 'precip_' + Xtest["hour"].astype(str)

In [ ]:
ff = Xtest.pivot(index='Id',columns='ff_idx',values='ff')
t = Xtest.pivot(index='Id',columns='t_idx',values='t')
td = Xtest.pivot(index='Id',columns='td_idx',values='td')
hu = Xtest.pivot(index='Id',columns='hu_idx',values='hu')
dd = Xtest.pivot(index='Id',columns='dd_idx',values='dd')
precip = Xtest.pivot(index='Id',columns='precip_idx',values='precip')

In [ ]:
X_reshape = pd.concat([ff ,t,td,hu,dd,precip],axis=1)

In [ ]:
X_reshape.reset_index(inplace=True)
X_reshape

In [ ]:
X = X_reshape

X['day'] = X['Id'].str.rpartition("_")[2]
X['day'] = X['day'].astype('int')
X['day'] = X['day'] % 365

In [ ]:
X['number_sta'] = X['Id'].str.rpartition('_')[0]
X['number_sta'] = X['number_sta'].astype('int64')

In [ ]:
X['season'] =pd.cut(X['day'],[0,60,151,243,334], labels=['Hiver','Printemps','Ete', 'Automne'],ordered=False)
X['season'] = X['season'].fillna('Hiver')
X['season']

In [ ]:
X['month'] = X['day'] / 30
X['month'] = X['month'].astype('int')

In [ ]:
X.reset_index(inplace=True)
X.drop('index',axis=1,inplace=True)

In [ ]:
X['month'] = X['month'].astype('category')
X['season'] = X['season'].astype('category')
X['day'] = X['day'].astype('category')

In [ ]:
X =pd.merge(X,coords,how='left',on='number_sta')
X['number_sta'] = X['number_sta'].astype('category')
X.fillna(method = 'bfill',inplace = True)

In [ ]:

X_test = transformer.fit_transform(X)
predictions_test = random_forest.predict(X_test)

In [ ]:
predictions_test

In [ ]:
index_pluie =np.where(predictions_test==2)

In [ ]:
X_pluie = X.loc[index_pluie]
X_pluie

In [ ]:
X_pluie_test = transformer.fit_transform(X_pluie)

In [ ]:
predictions_test_regressor = regressor.predict(X_pluie_test)

In [ ]:
predictions_test_regressor_quant = all_models['model_1'].predict(X_pluie_test)

In [ ]:
array_test = pd.DataFrame(predictions_test)
array_test[0] = np.where(array_test[0]==1,0,2)
array_test['Id'] = X['Id']
display(array_test)

In [ ]:
array_test.set_axis(['pred', 'Id'], axis=1, inplace=True)

In [ ]:
array_test

In [ ]:
array_test.pred.value_counts()

In [ ]:
tab_reg = {'pred' : predictions_test_regressor ,'Id' : X_pluie['Id']}
tab_reg = pd.DataFrame(tab_reg)
tab_reg

In [ ]:
tab_reg_quant = {'pred_quant' : predictions_test_regressor_quant ,'Id' : X_pluie['Id']}
tab_reg_quant = pd.DataFrame(tab_reg_quant)
tab_reg_quant

In [ ]:
pred = pd.merge(array_test,tab_reg,on ='Id',how='left')
pred

In [ ]:
pred['pred_y'] = pred.pred_y.fillna(0)
pred.drop('pred_x',axis=1,inplace=True)
pred.rename(columns = {'pred_y' : 'pred'},inplace=True)
pred

In [ ]:
pred = pd.merge(pred,tab_reg_quant,on ='Id',how='left')
pred

In [ ]:
pred['pred_quant'] = pred.pred_quant.fillna(0)
pred

In [ ]:
fname = './defi-ia-2022/Test/Test/Baselines/Baseline_observation_test.csv'
Baseline_observation_test = pd.read_csv(fname)
Baseline_observation_test = Baseline_observation_test.sort_values(by=["Id"])
display(Baseline_observation_test)

In [ ]:
pred = pd.merge(Baseline_observation_test,pred,on ='Id',how='left')
pred

In [ ]:
pred = pred.drop(["Prediction","pred"],axis=1)
pred

In [ ]:
pred.rename(columns = {'pred_quant' : 'Prediction'},inplace=True)
pred['Prediction']=pred['Prediction']+1
pred

In [ ]:
# Enregistrement prédictions

output_file = "prediction_test.csv"
pred.to_csv('./predictions/' + output_file,index=False)

Utilisons les données 2D

In [2]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

C:\Users\yassi\AppData\Local\Programs\Python\Python39\lib\site-packages\xarray\backends\cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


In [12]:
from netCDF4 import Dataset
import h5py

In [3]:
model = 'arome'  #weather model (arome or arpege)
param = "tp"   #parameter name in the file (cf cells below to know the parameter names -> exploration of metadata)
file_date = dt.datetime(2017, 2, 14) # Day example 

In [5]:
fname = "defi-ia-2022/Train/Train/X_forecast/2D_%s_%s.nc" %(model,file_date.strftime('%Y%m%d'))
#data = xr.load_dataset(fname)
#data

In [6]:
from scipy.io import netcdf
import netCDF4
nc = netCDF4.Dataset(fname)
nc.variables.keys()

dict_keys(['time', 'latitude', 'longitude', 'valid_time', 'ws', 'p3031', 'u10', 'v10', 't2m', 'd2m', 'r', 'tp', 'msl', 'Id'])

In [10]:
latitude = nc.variables['latitude']
latitude = nc.variables['latitude']
latitude = nc.variables['latitude']
latitude = nc.variables['latitude']
latitude = nc.variables['latitude']
latitude = nc.variables['latitude']
latitude = nc.variables['latitude']
latitude = nc.variables['latitude']
latitude = nc.variables['latitude']
latitude = nc.variables['latitude']
latitude = nc.variables['latitude']
latitude = nc.variables['latitude']

In [11]:
nc.variables['Id']

<class 'netCDF4._netCDF4.Variable'>
vlen Id(Id)
vlen data type: <class 'str'>
unlimited dimensions: 
current shape = (25,)